In [1]:
# Adapated from https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/ - Courtesy of SciPy
# Slider from -> https://matplotlib.org/3.1.1/gallery/widgets/slider_demo.html - Courtesty of Matplotlib
# UK COVID Data -> https://ourworldindata.org/coronavirus/country/united-kingdom?country=~GBR (OWID)

import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt, mpld3
from ipywidgets import interactive

cases = pd.read_csv('owid-covid-data.csv')
cases = cases[cases['iso_code']=='USA']
cases.drop(['life_expectancy','hospital_beds_per_thousand','handwashing_facilities','male_smokers','female_smokers','diabetes_prevalence','cardiovasc_death_rate','extreme_poverty','gdp_per_capita','aged_70_older', 'aged_65_older','median_age','population_density','population','stringency_index','tests_units','positive_rate','tests_per_case','new_tests_smoothed_per_thousand','new_tests_smoothed','new_tests_per_thousand','total_tests_per_thousand','total_tests','new_tests','new_deaths_per_million','total_deaths_per_million','new_cases_per_million','total_cases_per_million','new_deaths','total_deaths','new_cases','continent','location','new_deaths_smoothed_per_million','new_cases_smoothed_per_million','new_deaths_smoothed','new_cases_smoothed'],axis=1, inplace=True)
cases = cases[cases['date']=='2020-08-20']

pop = pd.read_csv('owid-covid-data.csv')
pop = pop[pop['iso_code']=='USA']
pop.drop(['life_expectancy','hospital_beds_per_thousand','handwashing_facilities','male_smokers','female_smokers','diabetes_prevalence','cardiovasc_death_rate','extreme_poverty','gdp_per_capita','aged_70_older', 'aged_65_older','median_age','population_density','stringency_index','tests_units','positive_rate','tests_per_case','new_tests_smoothed_per_thousand','new_tests_smoothed','new_tests_per_thousand','total_tests_per_thousand','total_tests','new_tests','new_deaths_per_million','total_deaths_per_million','new_cases_per_million','total_cases_per_million','new_deaths','total_deaths','new_cases','continent','location','total_cases', 'new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million'],axis=1, inplace=True)
pop = pop[pop['date']=='2020-08-20']

N = pop['population']
I0, R0 = cases['total_cases'], 0
S0 = N - I0 - R0
beta, gamma = 0, 0
t = np.linspace(0, 60, 60)

# The SIR model differential equations.
def sir(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Initial conditions vector
y0 = S0, I0, R0

# Plot the data on three separate curves for S(t), I(t) and R(t)
def sir_interactive_func(beta, gamma):
    ret = odeint(sir, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    
    fig = plt.figure()
    ax = fig.add_subplot(111, axisbelow=True)
    ax.plot(t, S/1000, 'yellow', lw=1.5, label='Susceptible')
    ax.plot(t, I/1000, 'red', lw=1.5, label='Infected')
    ax.plot(t, R/1000, 'blue', lw=1.5, label='Recovered')
    ax.set_xlabel('Time (days)')
    ax.set_ylabel('Population (1000s)')
    ax.grid(b=True, which='major', c='#bbbbbb', lw=1, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    #mpld3.save_html(fig, 'usa.html')

interactive_plot = interactive(sir_interactive_func, beta=(0.10,2,0.01), gamma=(0.10,1,0.01))
interactive_plot

interactive(children=(FloatSlider(value=1.05, description='beta', max=2.0, min=0.1, step=0.01), FloatSlider(va…